In [2]:
import networkx as nx
from smart_open import open
import ujson
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFMCS
import os
from xenosite.fragment.remap import serialize_and_remap
from rdkit.Chem.rdFMCS import FindMCS

from xenopict import Xenopict
import xenopict.magic

from rdkit.Chem import rdDepictor
import rdkit.Chem.rdDepictor 
from rdkit.Chem.TemplateAlign import AlignMolToTemplate2D

from xenosite.fragment.chem import Fragment

import copy
import tqdm
from rdkit import Chem
from typing import NamedTuple, Callable

import smart_open
import pickle
from collections import defaultdict

import algorithmx
from random import randint

from pyvis.network import Network

# Load in Reactivity Dataset

In [3]:
# Reactivity data 
def load_reactivity_data(pathname: str):
    with open(pathname) as f:
        reactivity_data = ujson.load(f)

    for mol in tqdm.tqdm(reactivity_data["data"]):
        rids = mol['rid']

        try:
            serial, rids = serialize_and_remap(mol['smiles'], rids)
        except:
            continue
        
        mol['smiles'] = serial.string
        mol['rid'] = rids

    return reactivity_data

SOMS = load_reactivity_data(os.environ["FRAGMENT_DATA_DIR"] + '/bioact_data.json')

100%|██████████| 1301/1301 [00:00<00:00, 5385.76it/s]


# Load Molecules Family Tree

In [4]:
class AMDLoadingData(NamedTuple):
  graph : nx.DiGraph
  parent_errors : list[str]
  child_errors : list[str]

# Parents and children
# parents larger than children
def load_AMD(pathname) -> AMDLoadingData:
  with open(pathname, 'rb') as f:
    raw_AMD = ujson.load(f)

  G = nx.DiGraph()

  parent_errors = []
  child_errors =[]

  for node in tqdm.tqdm(raw_AMD['data']):
    if not (mol := Chem.MolFromSmiles(node["smiles"])):
      parent_errors.append(node["smiles"])
      continue

    smi = Chem.MolToSmiles(mol, isomericSmiles=False, canonical=True)

    G.add_node(smi)
    G.nodes[smi]['levels'] = []
    G.nodes[smi]['rids'] = []

    for child in node["children"]:
      if not (mol := Chem.MolFromSmiles(child)):
        child_errors.append(child)
        continue
      
      child_smi = Chem.MolToSmiles(mol,isomericSmiles=False, canonical=True)

      if smi != child_smi:
        G.add_edge(smi, child_smi)
    
  return AMDLoadingData(G, parent_errors, child_errors)

_AMD = load_AMD(os.environ["FRAGMENT_DATA_DIR"] + "/amd_data.json.gz")

100%|██████████| 67159/67159 [00:23<00:00, 2892.29it/s]


In [5]:
AMD = copy.deepcopy(_AMD)

## Ancestor Traversal

In [6]:
def bfs_ancestor_edges(graph: nx.DiGraph, node):
    for child, parent in nx.bfs_edges(graph.reverse(), source=node):
        yield parent, child

def bfs_ancestors(graph: nx.DiGraph, node):
    for parent, child in bfs_ancestor_edges(graph, node):
        if parent == child: continue
        yield parent

def test_bfs_ancestors(graph):
    for node in graph.nodes:

        ancestors = set(bfs_ancestors(graph, node)) - set(node[smiles])

        for ancestor in ancestors:
            ancestor_ancestors = set(bfs_ancestors(graph, ancestor))  - set([ancestor])
            if not ancestor_ancestors: continue
            assert not (ancestor_ancestors - ancestors)
            return
    
    assert False, "Could not find any grandparents!"

def ancestor_generations(graph: nx.DiGraph, node) -> defaultdict:
    generation : dict[str, generation]= {}
    generation[node] = 0

    for parent, child in bfs_ancestor_edges(graph, node):
        if parent in generation: continue
        
        generation[parent] = generation[child] + 1

    output = defaultdict(list)

    for node, gen in generation.items():
        output[gen].append(node)

    return output


def get_children(graph, mol):
    return list(AMD.graph.successors(mol))

In [14]:
def display_rid_family_line(graph, propogated_rids, mol):
    df = pd.DataFrame(columns=['mol', 'mol image','rids', 'levels'])

    ag = list(ancestor_generations(graph, mol).items())

    for key, value in ag:
        for v in value:
            df.loc[len(df)] = [v, Xenopict(v).mark_atoms(propogated_rids[mol][v]['rids']), propogated_rids[mol][v]['rids'], propogated_rids[mol][v]['levels']]

    return df

out = display_rid_family_line(AMD.graph, prid, 'c1ccccc1')
out.style

## Cache Functions to Improve Performance

Learned this by coding with Josh.


In [25]:
output_data : dict[str, dict] = {}

In [11]:
# Caches for following computation
def cache_function_call(func):
    cache = {}

    def wrapper(*args, **kwargs):
        key = args
        if key in cache: return cache[key]
        else: 
            output = func(*args, **kwargs)
            cache[key] = output
            return output
    
    wrapper.cache = cache

    return wrapper

@cache_function_call
def get_rdkit_mol(smi):
    return Chem.MolFromSmiles(smi)

def match_molecules(*mols):
  return FindMCS(mols, bondCompare=Chem.rdFMCS.BondCompare.CompareAny)

@cache_function_call
def get_matches(parent, child):
    child_mol = get_rdkit_mol(child)
    parent_mol = get_rdkit_mol(parent)    
    patt = match_molecules(parent_mol, child_mol).queryMol 

    pm = parent_mol.GetSubstructMatches(patt)
    cm = child_mol.GetSubstructMatches(patt)

    if not pm: return []
    if not cm: return []

    out = []
    for p in pm:
        for c in cm:
            out.append((p,c))
    
    return out

## Propogating RIDs Through AMD

In [9]:
AMD = copy.deepcopy(_AMD)
# output_data : dict[str, dict] = {}

def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

def check_for_topological_equivalence(mol, rids):
    topological_rid_indices = []

    canonical_rank = list(Chem.rdmolfiles.CanonicalRankAtoms(mol, breakTies=False))

    for r in rids:
        get = canonical_rank[r]
        
        topological_rid_indices.extend(find_indices(canonical_rank, get))

    # rids.extend(topological_rid_indices)

    return topological_rid_indices

def check_for_topological_equivalence_one(mol, rid):
    topological_rid_indices = []

    canonical_rank = list(Chem.rdmolfiles.CanonicalRankAtoms(mol, breakTies=False))

    get = canonical_rank[rid]

    topological_rid_indices.extend(find_indices(canonical_rank, get))

    return topological_rid_indices[1:]

def multi_check(mol,rid_list):
    out = []
    for each in rid_list:
        out.extend(check_for_topological_equivalence_one(mol, each))

    return out

#  Add top eqv here for each parent that gets matched and initially
def propogate_rids(GRAPH : nx.DiGraph, smi : str, rids : list[int], get_matches : Callable):
    # Expand with top rids and levels
    rids = check_for_topological_equivalence(Chem.MolFromSmiles(smi), list(rids))
    PROPOGATED_RIDS : dict[str, dict[str, list[int]]] = {smi : {"rids": rids, "levels": [0] * len(rids)}}

    if not smi in GRAPH.nodes: return PROPOGATED_RIDS

    for parent, child in bfs_ancestor_edges(GRAPH, smi):

        rids, levels = [], []

        if child not in PROPOGATED_RIDS:
            continue
        
        if not PROPOGATED_RIDS[child]["rids"]:
            continue

        for parent_match, child_match in get_matches(parent, child):
            assert len(parent_match ) == len(child_match)

            # Create dictionary with to show the matched atom with its index in the match
            child_id_lookup = dict((i, index) for index, i in enumerate(child_match))

            lvl = []
            for r,l in zip(PROPOGATED_RIDS[child]["rids"], PROPOGATED_RIDS[child]["levels"]):
                if r in child_id_lookup:
                    parent_id = parent_match[child_id_lookup[r]]
                    
                    if parent not in PROPOGATED_RIDS:
                        PROPOGATED_RIDS[parent] = {"rids": [], "levels": []}
                        
                    # expand top rids and levels
                    top = check_for_topological_equivalence_one(Chem.MolFromSmiles(parent), parent_id)

                    PROPOGATED_RIDS[parent]["rids"].append(parent_id)
                    PROPOGATED_RIDS[parent]["rids"].extend(top)

                    PROPOGATED_RIDS[parent]["levels"].append(l+1)

    for each in PROPOGATED_RIDS:
        if len(PROPOGATED_RIDS[each]['rids']) != len(PROPOGATED_RIDS[each]['levels']):
            while len(PROPOGATED_RIDS[each]['rids']) != len(PROPOGATED_RIDS[each]['levels']):
                PROPOGATED_RIDS[each]['levels'].append(PROPOGATED_RIDS[each]['levels'][0])
            # print(PROPOGATED_RIDS[each])

    return  PROPOGATED_RIDS

def append_propogated_ids(GRAPH, mol, PROPOGATED_RIDS):
        GRAPH.graph.nodes[mol]["levels"].extend(PROPOGATED_RIDS[mol]["levels"])
        GRAPH.graph.nodes[mol]["rids"].extend(PROPOGATED_RIDS[mol]["rids"])

ERROR_MOLS = []
for mol in tqdm.tqdm(SOMS["data"]):
    rid = mol['rid']
    smi = mol['smiles']
 
    if smi not in output_data:
        try:
            output_data[smi] = propogate_rids(AMD.graph, smi, list(rid), get_matches)
        except Exception:
            ERROR_MOLS.append(smi)

  0%|          | 3/1301 [00:01<14:01,  1.54it/s]


KeyboardInterrupt: 

## Fixing Topological?

In [26]:
AMD = copy.deepcopy(_AMD)

def find_indices(list_to_check, item_to_find):
    return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

def topological_equiv_sites(mol, rids):
    canonical_rank = list(Chem.rdmolfiles.CanonicalRankAtoms(mol, breakTies=False))
    cr = set(canonical_rank[rid] for rid in rids)
    return [idx for idx, value in enumerate(canonical_rank) if value in cr]

#  Add top eqv here for each parent that gets matched and initially
def propogate_rids(GRAPH : nx.DiGraph, smi : str, rids : list[int], get_matches : Callable):
    # Expand with top rids and levels
    rids = topological_equiv_sites(Chem.MolFromSmiles(smi), list(rids))
    PROPOGATED_RIDS : dict[str, dict[str, list[int]]] = {smi : {"rids": rids, "levels": [0] * len(rids)}}

    if not smi in GRAPH.nodes: return PROPOGATED_RIDS

    for parent, child in bfs_ancestor_edges(GRAPH, smi):

        rids, levels = [], []

        if child not in PROPOGATED_RIDS:
            continue
        
        if not PROPOGATED_RIDS[child]["rids"]:
            continue

        for parent_match, child_match in get_matches(parent, child):
            assert len(parent_match ) == len(child_match)

            # Create dictionary with to show the matched atom with its index in the match
            child_id_lookup = dict((i, index) for index, i in enumerate(child_match))

            lvl = []
            for r,l in zip(PROPOGATED_RIDS[child]["rids"], PROPOGATED_RIDS[child]["levels"]):
                if r in child_id_lookup:
                    parent_id = parent_match[child_id_lookup[r]]
                    
                    if parent not in PROPOGATED_RIDS:
                        PROPOGATED_RIDS[parent] = {"rids": [], "levels": []}
                        
                    # expand top rids and levels
                    rids = topological_equiv_sites(Chem.MolFromSmiles(parent), [parent_id])

                    PROPOGATED_RIDS[parent]["rids"].extend(rids)
                    PROPOGATED_RIDS[parent]["levels"].extend([l+1] * len(rids))

    
    for smi in PROPOGATED_RIDS:
        rids = PROPOGATED_RIDS[smi]["rids"]
        levels = PROPOGATED_RIDS[smi]["levels"]

        nr_rids_levels = list(set(zip(rids, levels)))

        PROPOGATED_RIDS[smi]["rids"] = [r for r,l in nr_rids_levels]
        PROPOGATED_RIDS[smi]["levels"] = [l for r,l in nr_rids_levels]

    return  PROPOGATED_RIDS

def append_propogated_ids(GRAPH, mol, PROPOGATED_RIDS):
        GRAPH.graph.nodes[mol]["levels"].extend(PROPOGATED_RIDS[mol]["levels"])
        GRAPH.graph.nodes[mol]["rids"].extend(PROPOGATED_RIDS[mol]["rids"])


ERROR_MOLS = []
for mol in tqdm.tqdm(SOMS["data"]):
    rid = mol['rid']
    smi = mol['smiles']
 
    if smi not in output_data:
        try:
            output_data[smi] = propogate_rids(AMD.graph, smi, list(rid), get_matches)
        except Exception:
            ERROR_MOLS.append(smi)

100%|██████████| 1301/1301 [20:33<00:00,  1.05it/s]


In [28]:
len(output_data)

1298

In [30]:
processed_output = {}

def dataset_processing(output_data, processed_output):
    for smi in output_data:
        data = output_data[smi]

        for mol in data:
            if mol not in processed_output:
                processed_output[mol] = {"rids": [], "levels": []}

            processed_output[mol]['rids'].extend(data[mol]["rids"])
            processed_output[mol]['levels'].extend(data[mol]["levels"])

    return processed_output

def nr_processing(processed_output):
    for smi in processed_output:
        rids = processed_output[smi]["rids"]
        levels = processed_output[smi]["levels"]

        nr_rids_levels = list(sorted(set(zip(levels, rids))))

        processed_output[smi]["rids"] = [r for l,r in nr_rids_levels]
        processed_output[smi]["levels"] = [l for l,r in nr_rids_levels]

    return processed_output

nr_processing(dataset_processing(output_data, processed_output))
len(processed_output)

20957

In [32]:
with smart_open.open("propogated_bioactivation_set_dict.pkl.gz", 'wb') as f:
    pickle.dump(processed_output, f)

In [31]:
processed_output

{'O=C(Br)C(F)(F)F': {'rids': [1], 'levels': [0]},
 'FC(F)(F)C(Cl)Br': {'rids': [4], 'levels': [1]},
 'CC(C)C(Br)C(=O)NC(N)=O': {'rids': [3, 3], 'levels': [0, 1]},
 'CC(C)C(Br)C(=O)O': {'rids': [3], 'levels': [0]},
 'BrC1=CC(Br)=C(Oc2cc(Br)c(Br)cc2Br)C2OC12': {'rids': [16, 18],
  'levels': [0, 0]},
 'Brc1ccc(Oc2cc(Br)c(Br)cc2Br)c(Br)c1': {'rids': [2,
   3,
   4,
   6,
   7,
   8,
   12,
   17,
   2,
   3,
   7,
   8,
   12,
   17,
   1,
   2,
   3,
   4,
   6,
   7,
   8,
   10,
   12,
   13,
   15,
   17,
   1,
   2,
   3,
   4,
   6,
   7,
   8,
   10,
   12,
   13,
   15,
   17],
  'levels': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   2,
   2,
   2,
   2,
   2,
   2,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   3,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4,
   4]},
 'BrC1=CC(Br)=C(Oc2ccc(Br)cc2Br)C2OC12': {'rids': [9, 15], 'levels': [0, 0]},
 'Brc1ccc(Oc2ccc(Br)cc2Br)c(Br)c1': {'rids': [2, 3, 7, 8],
  'levels': [1, 1, 1, 1]},
 'O=C1C

In [35]:
Chem.MolFromSmarts(str(Fragment('O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]'))).GetNumAtoms()

6

In [56]:
len(test_df)

873

In [49]:
old_sa_set

alert_id                 alert_name  \
0           1  R1 Reactive alkyl halides   
1           2            R2 Acid halides   
2           3              R3 Carbazides   
3           4         R4 Sulphate esters   
4           5             R5 Sulphonates   
..        ...                        ...   
931      1099                    saponin   
932      1100                   monensin   
933      1101               squalestatin   
934      1102                   cyanidin   
935      1103               cytochalasin   

                                                smarts set_name  
0                                [Br,Cl,I][CX4;CH,CH2]    Glaxo  
1                             [S,C](=[O,S])[F,Br,Cl,I]    Glaxo  
2                                       O=CN=[N+]=[N-]    Glaxo  
3                                        COS(=O)O[C,c]    Glaxo  
4                                     COS(=O)(=O)[C,c]    Glaxo  
..                                                 ...      ...  
931                            O1CCCCC1OC2CCC3CCCCC3C2    MLSMR  
932                                    O1CCCCC1C2CCCO2    MLSMR  
933                                     C12OCCC(O1)CC2    MLSMR  
934  [OH]c1cc([OH])cc2=[O+]C(=C([OH])Cc21)c3cc([OH]...    MLSMR  
935                                    O=C1NCC2CCCCC21    MLSMR  

[936 rows x 4 columns]

In [37]:
import gzip 

with gzip.open('alerts_stats.pkl.gz', 'rb') as f:
    test = pickle.load(f)

test_df = test.to_pandas()

test_df

count  marked_count  \
frag                                                                          
[S,C](=[O,S])[F,Br,Cl,I]                            26.666667           6.0   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]         22.666667           5.0   
C(=O)[Cl,Br,I,F]                                    18.333333           5.0   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)         8.800000           2.0   
O=C[F,Cl,Br,I]                                      18.333333           5.0   
...                                                       ...           ...   
[#8]=[#6]-4-[#6]-[#6]-[#6]-3-[#6]-2-[#6](=[#8])...   6.000000           0.0   
[CH1]#CC(=O)-[#6,#7,#8]                              1.000000           0.0   
O=COC=[$(C(S(=O)(=O))),$(C(C(F)(F)(F))),$(C(C#N...   1.000000           0.0   
[S;!R]-[C;!R](=[S;!R])(-[S;!R])                      1.000000           0.0   
[C;H2](F)C(=O)[O,N,S]                                1.000000           0.0   

                                                    n_mol  n_atom  n_mark  \
frag                                                                        
[S,C](=[O,S])[F,Br,Cl,I]                               25     175       6   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]            22     212      25   
C(=O)[Cl,Br,I,F]                                       18     138       5   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)            8      79       2   
O=C[F,Cl,Br,I]                                         18     138       5   
...                                                   ...     ...     ...   
[#8]=[#6]-4-[#6]-[#6]-[#6]-3-[#6]-2-[#6](=[#8])...      6     145       0   
[CH1]#CC(=O)-[#6,#7,#8]                                 1       5       0   
O=COC=[$(C(S(=O)(=O))),$(C(C(F)(F)(F))),$(C(C#N...      1      37       0   
[S;!R]-[C;!R](=[S;!R])(-[S;!R])                         1      23       0   
[C;H2](F)C(=O)[O,N,S]                                   1       5       0   

                                                    n_cover  n_mark_cover  \
frag                                                                        
[S,C](=[O,S])[F,Br,Cl,I]                                 80             6   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]             136             7   
C(=O)[Cl,Br,I,F]                                         55             5   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)             44             2   
O=C[F,Cl,Br,I]                                           55             5   
...                                                     ...           ...   
[#8]=[#6]-4-[#6]-[#6]-[#6]-3-[#6]-2-[#6](=[#8])...      114             0   
[CH1]#CC(=O)-[#6,#7,#8]                                   5             0   
O=COC=[$(C(S(=O)(=O))),$(C(C(F)(F)(F))),$(C(C#N...        5             0   
[S;!R]-[C;!R](=[S;!R])(-[S;!R])                           4             0   
[C;H2](F)C(=O)[O,N,S]                                     5             0   

                                                         exp  obs  \
frag                                                                
[S,C](=[O,S])[F,Br,Cl,I]                            3.182143    6   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]              NaN    5   
C(=O)[Cl,Br,I,F]                                    2.582143    5   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)        1.428571    2   
O=C[F,Cl,Br,I]                                      2.582143    5   
...                                                      ...  ...   
[#8]=[#6]-4-[#6]-[#6]-[#6]-3-[#6]-2-[#6](=[#8])...  0.000000    0   
[CH1]#CC(=O)-[#6,#7,#8]                             0.000000    0   
O=COC=[$(C(S(=O)(=O))),$(C(C(F)(F)(F))),$(C(C#N...  0.000000    0   
[S;!R]-[C;!R](=[S;!R])(-[S;!R])                     0.000000    0   
[C;H2](F)C(=O)[O,N,S]                               0.000000    0   

                                                                                           marked_ids  
frag                  

In [57]:
with smart_open.open("full_alert_stats.pkl.gz", 'wb') as f:
    pickle.dump(test_df, f)

In [59]:
with gzip.open('full_alert_stats.pkl.gz', 'rb') as f:
    test1 = pickle.load(f)

test1

count  marked_count  \
frag                                                                          
[S,C](=[O,S])[F,Br,Cl,I]                            26.666667           6.0   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]         22.666667           5.0   
C(=O)[Cl,Br,I,F]                                    18.333333           5.0   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)         8.800000           2.0   
O=C[F,Cl,Br,I]                                      18.333333           5.0   
...                                                       ...           ...   
c1ccccc1-n2nnnc2[CH2]*                               0.000000           0.0   
N#CC(=C)C#N                                          0.000000           0.0   
a-P(-a)-a                                            0.000000           0.0   
[#6]OP(=O)(*)O[#6].[#6]OP(=O)(*)O[#6].[#6]OP(=O...   0.000000           0.0   
c12cccc(C(=O)N(-&!@C)C(=O)3)c2c3ccc1                 0.000000           0.0   

                                                    n_mol  n_atom  n_mark  \
frag                                                                        
[S,C](=[O,S])[F,Br,Cl,I]                               25     175       6   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]            22     212      25   
C(=O)[Cl,Br,I,F]                                       18     138       5   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)            8      79       2   
O=C[F,Cl,Br,I]                                         18     138       5   
...                                                   ...     ...     ...   
c1ccccc1-n2nnnc2[CH2]*                                  0       0       0   
N#CC(=C)C#N                                             0       0       0   
a-P(-a)-a                                               0       0       0   
[#6]OP(=O)(*)O[#6].[#6]OP(=O)(*)O[#6].[#6]OP(=O...      0       0       0   
c12cccc(C(=O)N(-&!@C)C(=O)3)c2c3ccc1                    0       0       0   

                                                    n_cover  n_mark_cover  \
frag                                                                        
[S,C](=[O,S])[F,Br,Cl,I]                                 80             6   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]             136             7   
C(=O)[Cl,Br,I,F]                                         55             5   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)             44             2   
O=C[F,Cl,Br,I]                                           55             5   
...                                                     ...           ...   
c1ccccc1-n2nnnc2[CH2]*                                    0             0   
N#CC(=C)C#N                                               0             0   
a-P(-a)-a                                                 0             0   
[#6]OP(=O)(*)O[#6].[#6]OP(=O)(*)O[#6].[#6]OP(=O...        0             0   
c12cccc(C(=O)N(-&!@C)C(=O)3)c2c3ccc1                      0             0   

                                                         exp  obs  \
frag                                                                
[S,C](=[O,S])[F,Br,Cl,I]                            3.182143    6   
O=CC(-[F,Cl,Br,I])([F,Cl,Br,I])-[F,Cl,Br,I]              NaN    5   
C(=O)[Cl,Br,I,F]                                    2.582143    5   
[$(C(=O));!$(C-N);!$(C-O);!$(C-S)]C(F)(F)(F)        1.428571    2   
O=C[F,Cl,Br,I]                                      2.582143    5   
...                                                      ...  ...   
c1ccccc1-n2nnnc2[CH2]*                              0.000000    0   
N#CC(=C)C#N                                         0.000000    0   
a-P(-a)-a                                           0.000000    0   
[#6]OP(=O)(*)O[#6].[#6]OP(=O)(*)O[#6].[#6]OP(=O...  0.000000    0   
c12cccc(C(=O)N(-&!@C)C(=O)3)c2c3ccc1                0.000000    0   

                                                                                           marked_ids  
frag                  

In [29]:
old_set = pd.read_csv('/Users/clint/Desktop/xenosite-fragment/bioactivation_dataset.csv', sep='\t', header=None)

for mol in list(old_set[0]):
    if mol not in output_data:
        output_data[mol] = {mol: {"rids": [], "levels": []}}

len(output_data)

20957

In [16]:
output_data

{'O=C(Br)C(F)(F)F': {'O=C(Br)C(F)(F)F': {'rids': [1], 'levels': [0]},
  'FC(F)(F)C(Cl)Br': {'rids': [4], 'levels': [1]}},
 'CC(C)C(Br)C(=O)NC(N)=O': {'CC(C)C(Br)C(=O)NC(N)=O': {'rids': [3],
   'levels': [0]}},
 'CC(C)C(Br)C(=O)O': {'CC(C)C(Br)C(=O)O': {'rids': [3], 'levels': [0]},
  'CC(C)C(Br)C(=O)NC(N)=O': {'rids': [3], 'levels': [1]}},
 'BrC1=CC(Br)=C(Oc2cc(Br)c(Br)cc2Br)C2OC12': {'BrC1=CC(Br)=C(Oc2cc(Br)c(Br)cc2Br)C2OC12': {'rids': [18,
    16],
   'levels': [0, 0]},
  'Brc1ccc(Oc2cc(Br)c(Br)cc2Br)c(Br)c1': {'rids': [3, 2], 'levels': [1, 1]}},
 'BrC1=CC(Br)=C(Oc2ccc(Br)cc2Br)C2OC12': {'BrC1=CC(Br)=C(Oc2ccc(Br)cc2Br)C2OC12': {'rids': [9,
    15],
   'levels': [0, 0]},
  'Brc1ccc(Oc2ccc(Br)cc2Br)c(Br)c1': {'rids': [3, 7, 2, 8],
   'levels': [1, 1, 1, 1]},
  'Brc1ccc(Oc2cc(Br)c(Br)cc2Br)c(Br)c1': {'rids': [8, 3, 7, 2],
   'levels': [2, 2, 2, 2]}},
 'O=C1C=CC(Br)=CC1=O': {'O=C1C=CC(Br)=CC1=O': {'rids': [2], 'levels': [0]},
  'Brc1ccccc1': {'rids': [3, 5], 'levels': [1, 1]},
  'Oc1ccc(B

In [14]:
a = pd.read_csv('/Users/clint/Desktop/xenosite-fragment/propogated_bioactivation_dataset.csv', sep='\t')
# len(a)
a

FileNotFoundError: [Errno 2] No such file or directory: '/Users/clint/Desktop/xenosite-fragment/propogated_bioactivation_dataset.csv'

In [79]:
# smi="BrC1=CC(Br)=C(Oc2cc(Br)c(Br)cc2Br)C2OC12"
# rids = processed_output[smi]["rids"]
# levels = processed_output[smi]["levels"]


def dataset_to_csv(dataset, filename):
    text = ''

    for smi in dataset:
        rids = dataset[smi]["rids"]
        levels = dataset[smi]["levels"]

        # print("%s\t%s" % ( smi, str(list(zip(rids, levels)))))

        text += "%s\t%s\n" % ( smi, str(list(zip(rids, levels))))

    with open(filename, 'w') as f:
        f.write(text)

# text

dataset_to_csv(processed_output, 'new_propogated_bioactivation_dataset.csv')

KeyError: 'rids'

In [24]:
with smart_open.open("/Users/clint/Desktop/xenosite-fragment/propogated_bioactivation_dataset.pkl.gz", 'rb') as f:
    tprid = pickle.load(f)

tprid

'O=C(Br)C(F)(F)F\t[(1, 0)]\nFC(F)(F)C(Cl)Br\t[(4, 1)]\nCC(C)C(Br)C(=O)NC(N)=O\t[(3, 0), (3, 1)]\nCC(C)C(Br)C(=O)O\t[(3, 0)]\nBrC1=CC(Br)=C(Oc2cc(Br)c(Br)cc2Br)C2OC12\t[(16, 0), (18, 0)]\nBrc1ccc(Oc2cc(Br)c(Br)cc2Br)c(Br)c1\t[(2, 1), (3, 1), (4, 1), (6, 1), (7, 1), (8, 1), (12, 1), (17, 1), (2, 2), (3, 2), (7, 2), (8, 2), (12, 2), (17, 2), (1, 3), (2, 3), (3, 3), (4, 3), (6, 3), (7, 3), (8, 3), (10, 3), (12, 3), (13, 3), (15, 3), (17, 3), (1, 4), (2, 4), (3, 4), (4, 4), (6, 4), (7, 4), (8, 4), (10, 4), (12, 4), (13, 4), (15, 4), (17, 4)]\nBrC1=CC(Br)=C(Oc2ccc(Br)cc2Br)C2OC12\t[(9, 0), (15, 0)]\nBrc1ccc(Oc2ccc(Br)cc2Br)c(Br)c1\t[(2, 1), (3, 1), (7, 1), (8, 1)]\nO=C1C=CC(Br)=CC1=O\t[(2, 0)]\nBrc1ccccc1\t[(1, 0), (3, 0), (4, 0), (5, 0), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (2, 3), (3, 3), (5, 3), (6, 3)]\nOc1ccc(Br)cc1\t[(1, 1), (2, 1), (3, 1), (4, 1), (6, 1), (7, 1), (2, 2), (3, 2), (6, 2), (7, 2)]\nCC(=O)NC(CSC1C=C(Br)C=CC1O)C(=O)O\t[(7,

In [519]:
AMD = copy.deepcopy(_AMD)

for values in tprid.values():
    for k,v in zip(values.keys(), values.values()):
        print(k,v)
        if k not in AMD.graph.nodes:
            AMD.graph.add_node(k)
            AMD.graph.nodes[k]['levels'] = []
            AMD.graph.nodes[k]['rids'] = []
        
        AMD.graph.nodes[k]["levels"].extend(v["levels"])
        AMD.graph.nodes[k]["rids"].extend(v["rids"])

O=C(Br)C(F)(F)F {'rids': [1], 'levels': [0]}
FC(F)(F)C(Cl)Br {'rids': [4], 'levels': [1]}
CC(C)C(Br)C(=O)NC(N)=O {'rids': [3], 'levels': [0]}
CC(C)C(Br)C(=O)O {'rids': [3], 'levels': [0]}
CC(C)C(Br)C(=O)NC(N)=O {'rids': [3], 'levels': [1]}
BrC1=CC(Br)=C(Oc2cc(Br)c(Br)cc2Br)C2OC12 {'rids': [16, 18], 'levels': [0, 0]}
Brc1ccc(Oc2cc(Br)c(Br)cc2Br)c(Br)c1 {'rids': [3, 2], 'levels': [1, 1]}
BrC1=CC(Br)=C(Oc2ccc(Br)cc2Br)C2OC12 {'rids': [15, 9], 'levels': [0, 0]}
Brc1ccc(Oc2ccc(Br)cc2Br)c(Br)c1 {'rids': [3, 7, 8, 8], 'levels': [1, 1, 1, 1]}
Brc1ccc(Oc2cc(Br)c(Br)cc2Br)c(Br)c1 {'rids': [3, 7], 'levels': [2, 2]}
O=C1C=CC(Br)=CC1=O {'rids': [2], 'levels': [0]}
Brc1ccccc1 {'rids': [3, 5], 'levels': [1, 1]}
Oc1ccc(Br)cc1 {'rids': [2, 7], 'levels': [1, 1]}
CC(=O)NC(CSC1C=C(Br)C=CC1O)C(=O)O {'rids': [12], 'levels': [2]}
OC1C=CC(Br)=CC1O {'rids': [2], 'levels': [2]}
Brc1cccc(Br)c1 {'rids': [3, 3], 'levels': [2, 2]}
Brc1ccccc1Br {'rids': [3, 4, 4, 4], 'levels': [2, 2, 2, 2]}
COC(=O)C(C)NP(=O)(OCC1C=C

In [526]:
len(_AMD.graph.nodes)

67158

In [520]:
with smart_open.open("AMD_network_with_propogated_rids.pkl.gz", 'wb') as f:
    pickle.dump(AMD.graph, f)

## Loading and Saving Propogated IDS

In [499]:
# with smart_open.open("correct_topological_propogated_reactive_ids.pkl.gz", 'wb') as f:
#     pickle.dump(output_data, f)

In [8]:
with smart_open.open("propogated_reactive_ids.pkl.gz", 'rb') as f:
    prid = pickle.load(f)